# 把stage0 -- contactgen -- stage1 -- stage2的结果串联起来，并计算指标

## stage0

In [1]:
import numpy as np
import torch
import pickle
import trimesh
from os.path import join
from tqdm import *

/root/anaconda3/envs/mdm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = 'final_results/stage0_results.npy'
stage0_data = np.load(path,allow_pickle=True).item()

In [3]:
stage0_data.keys()

dict_keys(['motion', 'lengths', 'seqs', 'num_samples', 'num_repetitions', 'all_gt'])

In [4]:
seqs = stage0_data['seqs']
print(len(seqs))
set_seqs = set(seqs)
print(len(set_seqs))

81
81


In [5]:
motion = stage0_data['motion']
print(motion.shape)

(81, 69, 3, 4)


### 插值 恢复30fps帧率  6--30

In [6]:
import torch
import numpy as np
from pytorch3d.transforms import rotation_conversions as rc

def slerp(q0, q1, t):
    dot = torch.dot(q0, q1)
    if dot < 0.0:
        q1 = -q1
        dot = -dot
    DOT_THRESHOLD = 0.9995
    if dot > DOT_THRESHOLD:
        result = q0 + t * (q1 - q0)
        return result / result.norm()
    theta_0 = dot.acos()
    sin_theta_0 = theta_0.sin()
    theta = theta_0 * t
    sin_theta = theta.sin()
    s0 = ((1.0 - t) * theta).cos()
    s1 = sin_theta / sin_theta_0
    return (s0 * q0) + (s1 * q1)

def interpolate_rotations_and_translations(rot_matrices, translations, timestamps_original, timestamps_target):
    # 将旋转矩阵转换为四元数
    quaternions = rc.matrix_to_quaternion(rot_matrices)
    
    # 插值结果容器
    quaternions_interpolated = []
    translations_interpolated = []

    for i in range(len(timestamps_target)):
        # 计算当前目标时间戳在原始时间戳的位置
        t_norm = timestamps_target[i] * (len(timestamps_original) - 1)
        idx = int(t_norm)
        t = t_norm - idx

        if idx < len(timestamps_original) - 1:
            # 对四元数进行SLERP插值
            q_interp = slerp(quaternions[idx], quaternions[idx + 1], t)
            quaternions_interpolated.append(q_interp)

            # 对平移向量进行线性插值
            trans_interp = (1 - t) * translations[idx] + t * translations[idx + 1]
            translations_interpolated.append(trans_interp)
        else:
            # 直接使用最后一个四元数和平移向量
            quaternions_interpolated.append(quaternions[-1])
            translations_interpolated.append(translations[-1])

    # 将插值后的四元数转换回旋转矩阵
    quaternions_interpolated = torch.stack(quaternions_interpolated)
    rot_matrices_interpolated = rc.quaternion_to_matrix(quaternions_interpolated)
    
    # 将插值后的平移向量转换为合适的形式
    translations_interpolated = torch.stack(translations_interpolated)

    return rot_matrices_interpolated, translations_interpolated

# 示例使用
# 假设的旋转矩阵和平移向量
rot_matrices = torch.rand((69, 3, 3))  # [N, 3, 3]
translations = torch.rand((69, 3))     # [N, 3]

# 定义时间戳
timestamps_original = np.linspace(0, 1, 69)
timestamps_target = np.linspace(0, 1, 345)

# 执行插值
rot_matrices_interpolated, translations_interpolated = interpolate_rotations_and_translations(
    rot_matrices, translations, timestamps_original, timestamps_target
)

# 此时，`rot_matrices_interpolated` 和 `translations_interpolated` 包含了插值后的旋转矩阵和平移向量


In [7]:
rot_matrices_interpolated.shape

torch.Size([345, 3, 3])